In [39]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In this kernel I'm just going to do some missing value imputation, generate dummy variables when ever required and build some basic models and make a prediction based on models that give good accuracy that are also least correlated.

In [40]:
# Getting all the required packages
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('whitegrid')
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 

In [41]:
# Reading the train and test files
train_data = pd.read_csv('../input/train.csv')
test_data = pd.read_csv('../input/test.csv')

In [42]:
print('Shape of train data: ', train_data.shape)
print('Shape of test data: ', test_data.shape)

So, we are having 11 variables overall, the additional one in the train data is the output variable.
And there are 891 examples in the train dataset and there are 418 examples in the test dataset.
Generally for building any ML model (that has good accuracy and less variance) the amount of data requried should be at least in 1000s.

Generally for building any ML model (that has good accuracy and less variance) the amount of data requried should be at least in 1000s.

In [43]:
# getting a look at what the train data looks like

train_data.head()

1. By looking at the data all I can clearly see some useless variables - they are Name, Ticket, PassengerID and Cabin. 
2. Ticket and Cabin could be analysed using some NLP, however, I'm not going to do that.
3. So I'm going to remove those columns from both train and test

In [44]:
# Getting the overall description of the data that we have
train_data.info()
print('\n----------------------------------\n')
test_data.info()

* Age, Fare and Embarked has to be imputed in the train and test dataset.
* But, before that let's drop the unnecessary variables

In [45]:
train_data = train_data.drop(['Name', 'Ticket', 'PassengerId','Cabin'], axis =1)
test_data = test_data.drop(['Name', 'Ticket','Cabin'], axis = 1)

In [46]:
# Now its time for imputation
# Mode for categorical variable and mean for continuous variable
age_mean = train_data['Age'].mean()
train_data['Age'] = train_data['Age'].fillna(age_mean)
test_data['Age'] = test_data['Age'].fillna(age_mean)

fare_mean = train_data['Fare'].mean()
test_data['Fare'] = test_data['Fare'].fillna(fare_mean)

embarked_mode = train_data['Embarked'].mode()
train_data['Embarked'] = train_data['Embarked'].fillna(embarked_mode)
test_data['Embarked'] = test_data['Embarked'].fillna(embarked_mode)

* Now it's time to create the dummy variables for the dataset.
* I think we need dummy variables for Embarked, Sex and PClass
* But PClass is tagged as an int variable, looks like it has to be manually tagged as categorical variable

In [47]:
# Let's see what's there in the PClass
sns.distplot(train_data['Pclass'])

In [48]:
train_data.Pclass[train_data.Pclass == 1] = 'First'

train_data.Pclass[train_data.Pclass == 2] = 'Second'

train_data.Pclass[train_data.Pclass == 3] = 'Third'

test_data.Pclass[test_data.Pclass == 1] = 'First'

test_data.Pclass[test_data.Pclass == 2] = 'Second'

test_data.Pclass[test_data.Pclass == 3] = 'Third'

In [53]:
# Lets create the dummy variables now

train_dummy = pd.get_dummies(train_data)
test_dummy = pd.get_dummies(test_data)

In [50]:
train_dummy.info()
print('\n--------------------\n')
test_dummy.info()

* For a variable with k categories we only need (k-1) dummy variables.
* So let's remove the excess variables from both train and test

In [54]:
train_dummy.drop(['Pclass_First','Sex_male','Embarked_C'], axis =1, inplace = True)
test_dummy.drop(['Pclass_First','Sex_male','Embarked_C','PassengerId'], axis = 1, inplace = True)

In [55]:
Train_Y = train_dummy['Survived']
Train_X = train_dummy.drop(['Survived'], axis = 1)
Test_X = test_dummy

In [56]:
# Logistic Regression Model

LogReg = LogisticRegression()
LogReg.fit(Train_X, Train_Y)
Y_Pred = LogReg.predict(Test_X)
LogReg.score(Train_X, Train_Y)

In [57]:
# Support Vector Machines

svc = SVC()
svc.fit(Train_X, Train_Y)
Y_pred_svm = svc.predict(Test_X)
svc.score(Train_X, Train_Y)

In [58]:
# Random Forests

random_forest = RandomForestClassifier(n_estimators=100)

random_forest.fit(Train_X, Train_Y)

Y_pred_rf = random_forest.predict(Test_X)

random_forest.score(Train_X, Train_Y)

In [59]:
# Using KNN

knn = KNeighborsClassifier(n_neighbors = 1)

knn.fit(Train_X, Train_Y)

Y_pred_Knn = knn.predict(Test_X)

knn.score(Train_X, Train_Y)

In [60]:
# Gaussian Naive Bayes

gaussian = GaussianNB()

gaussian.fit(Train_X, Train_Y)

Y_pred_GNB = gaussian.predict(Test_X)

gaussian.score(Train_X, Train_Y)

In [61]:
Y = pd.DataFrame()
Y['Logistic'] = Y_Pred
Y['SVM'] = Y_pred_svm
Y['RF'] = Y_pred_rf
Y['KNN'] = Y_pred_Knn
Y['GNB'] = Y_pred_GNB

In [62]:
# Now lets see the correlation between all these models
Y.corr()

* It looks like none of the models are highly correlated.
* The three best models are RF, SVM and KNN.
* Lets make an ensemble of these three models.

In [63]:
Final_Pred = (Y_pred_rf + Y_pred_Knn + Y_pred_svm )/3
Final_Pred = np.round(Final_Pred)

In [64]:
submission = pd.DataFrame({
        "PassengerId": test_data["PassengerId"],
        "Survived": Final_Pred
    })

submission.to_csv('titanic.csv', index=False)